# APRIOIR ALGORITHM

### READING DATA

In [1]:
import pandas as pd
import time
import random

In [2]:
df=pd.read_excel("Shopping-cart-dataset.xlsx")

In [3]:
df.drop(['StockCode','Quantity','InvoiceDate','Price','Customer ID','Country'],axis=1,inplace=True)

In [4]:
df

,Invoice,Description
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.
...,...,...
50742,540551,VINTAGE ROSE BEAD BRACELET RASPBERR
50743,540551,PINK SWEETHEART BRACELET
50744,540551,LARGE CRYSTAL DIAMANTE HAIRSLIDE
50745,540551,GREEN ENAMEL+GLASS HAIR COMB


In [5]:
minimum_item_supports={}
for x in df["Description"].unique().tolist():
    minimum_item_supports[x]=50
for y in random.sample(minimum_item_supports.keys(),5):
    minimum_item_supports[x]=random.randint(30,70)

C:\Users\Hamza\AppData\Local\Temp\ipykernel_8900\3571172241.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for y in random.sample(minimum_item_supports.keys(),5):


In [6]:
import math
trans_df=df.groupby("Invoice").agg({"Description":lambda x:list(x)})
trans_df

,Description
Invoice,
536365,"[WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET..."
536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
536367,"[ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO..."
536368,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
536369,[BATH BUILDING BLOCK WORD]
...,...
C540417,"[CHILLI LIGHTS, TEA TIME PARTY BUNTING, PICNIC..."
C540470,"[WHITE TEA,COFFEE,SUGAR JARS]"
C540503,"[CAKE STAND VICTORIAN FILIGREE LARGE, 3 HEARTS..."


# FUNCTION TO FIND FREQUENT ITEMS FOR K=1

In [7]:
def calculate_frequent_items(data,sup):
    item_count={}
    for index,row in data.iterrows():
        for item in row["Description"]:
            if item in item_count.keys():
                item_count[item]+=1
            else:
                item_count[item]=1

    lis=[]
    for key,value in item_count.items():
        if value>=sup[key]:
            lis.append(key)
    return lis

### FINDING FREQUENT ITEMS FOR K==1 (NOTE: NO ITEMS JOINED OR PRUNED IN K==1)

In [8]:
print(calculate_frequent_items(trans_df,minimum_item_supports))

['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'HAND WARMER UNION JACK', 'ASSORTED COLOUR BIRD ORNAMENT', "POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN", 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'HOME BUILDING BLOCK WORD', 'LOVE BUILDING BLOCK WORD', 'DOORMAT NEW ENGLAND', 'JAM MAKING SET WITH JARS', 'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE GREEN', 'VINTAGE HEADS AND TAILS CARD GAME ', 'SET/2 RED RETROSPOT TEA TOWELS ', 'ROUND SNACK BOXES SET OF4 WOODLAND ', 'SPACEBOY LUNCH BOX ', 'LUNCH BOX I LOVE LONDON', 'RED TOADSTOOL LED NIGHT LIGHT', ' SET 2 TEA TOWELS I LOVE LONDON ', 'POSTAGE', "PAPER CHAIN KIT 50'S CHRISTMAS ", 'RETRO COFFEE MUGS ASSORTED', 'SAVE THE PLANET MUG', 'WOOD 2 DRAWER CABINET WHITE FINISH', 'WOOD S/3 CABINET ANT WHITE FINISH', 'WOODEN PICTURE FRAME WHITE FINISH', 'WOODEN

# GENERALIZATION FUNCTION TO FIND FREQUENT ITEMS FOR K>=2

In [9]:
from collections import Counter
def freq_items(data, sup):
    item_count = Counter()
    for itemset in data:
        A = set(tuple(itemset[0]))
        df = trans_df.loc[trans_df["Description"].apply(set).apply(A.issubset)]
        item_count.update({tuple(itemset[0]): df.shape[0]})
    lis=[]
    for itemset, count in item_count.items():
        minsup=[]
        for x in itemset:
            minsup.append(sup[x])
        if count >= min(minsup):
            lis.append("/".join(itemset))
    return lis
    # return ["/".join(itemset) for itemset, count in item_count.items() if count >= sup]

def join(data,k):
    if k==2:
        lis=[]
        lis_sub=[]
        lis_3=[]
        for i in range(0,len(data)-1):
            for j in range(i+1,len(data)):
                lis_sub=[]
                lis_3=[]
                lis_sub.append(data[i])
                lis_sub.append(data[j])
                lis_3.append(lis_sub)
                lis.append(lis_3)
        return lis
    else:
        lis=[]
        lis_3=[]
        for i in range(0,len(data)-1):
            x=data[i].split("/")
            for j in range(i+1,len(data)):
                y=data[j].split("/")
                lis_sub=[]
                lis_3=[]
                if x[:-1] == y[:-1] and x[-1] != y[-1]:
                    d=x.copy()
                    d.append(y[-1])
                    lis_3.append(d)
                    lis.append(lis_3)
        return lis

def prune(data,k,prev):
    pruned=[]
    if k==2:
        return data
    else:
        import itertools
        for i in range(0,len(data)):
            try:
                comb=list(itertools.combinations(data[i][0],k-1))
                length=len(comb)
                count=0
                for y in comb:
                    for z in prev:
                        if set(y)==set(z.split("/")):
                            count+=1
                if count != length:
                    pruned.append(data[i])
                    del data[i]
            except:
                pass
        print("Pruned itemsets: ")
        print(pruned)
        return data

        
def Generalization(dat,sup):
    k=2
    while True:
        candidate=prune(join(dat,k),k,dat)
        if len(candidate)==0 or k==6:
            return dat
        print("CANDIDATE ITEMS FOR K == ",k)
        print(candidate)
        
        temp=freq_items(candidate,sup)

        if len(temp)!=0:
            dat=temp
            print("QUALIFIED ITEMS FOR K == ",k)
            print(dat)
        else:
            print("NO ITEMS QUALIFIED K == ",k)
            return dat
        k+=1

### FINDING FREQUENT ITEMS (for k>=2)
### ALSO DISPLAYING CANDIDATE AND QUALIFIED CANDIDATES IN EACH Kth ITERATION

In [10]:
print(Generalization(calculate_frequent_items(trans_df,minimum_item_supports),minimum_item_supports))

CANDIDATE ITEMS FOR K ==  2
[[['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'CREAM CUPID HEARTS COAT HANGER']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'KNITTED UNION FLAG HOT WATER BOTTLE']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'RED WOOLLY HOTTIE WHITE HEART.']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'SET 7 BABUSHKA NESTING BOXES']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'HAND WARMER UNION JACK']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'ASSORTED COLOUR BIRD ORNAMENT']], [['WHITE HANGING HEART T-LIGHT HOLDER', "POPPY'S PLAYHOUSE BEDROOM "]], [['WHITE HANGING HEART T-LIGHT HOLDER', "POPPY'S PLAYHOUSE KITCHEN"]], [['WHITE HANGING HEART T-LIGHT HOLDER', 'FELTCRAFT PRINCESS CHARLOTTE DOLL']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'HOME BUILDING BLOCK WORD']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'LOVE BUILDING BLOCK WORD']], [['WHITE HANGING HEART T-LIGHT HOLDER', 'DOORMAT NEW ENGLAND']], [['WHITE HANGING HEART T-LIGH

# FUNCTION TO FIND CONFIDENCE OF FREQUENT ITEMS

In [11]:
import itertools

def confidence(data,conf,k):
    i=0
    item_count={}
    for itemsets in data:
        try:
            x=itemsets.split("/")
            A=set(x)
            for index,row in trans_df.iterrows():
                B=set(row["Description"])
                if B.intersection(A) == A:
                    if tuple(x) in item_count.keys():
                        item_count[tuple(x)]+=1
                    else:
                        item_count[tuple(x)]=1
        except:
            return [],[]
        
    lis=[]
    rule={}
    rule_name=[]
    rule_value=[]
    for key,value in item_count.items():
        for i in range(1,k):
            lis.append(list(itertools.combinations(key, i)))

        flat_list = list(itertools.chain(*lis))

        for i in range(0,len(flat_list)):
            for j in range(0,len(flat_list)):
                if i!=j and len(set(flat_list[i]).intersection(set(flat_list[j])))==0 and len(set(flat_list[j]).intersection(set(flat_list[i])))==0:
                    count=0
                    for index,row in trans_df.iterrows():
                        B=set(row["Description"])
                        if B.intersection(set(flat_list[i])) == set(flat_list[i]):
                            count+=1
                    if (item_count[key]/count)*100 >= conf:
                        rule_name.append(','.join(map(str, flat_list[i]))+" -> "+ ','.join(map(str, flat_list[j])))
                        rule_value.append((item_count[key]/count)*100)
        lis=[]
    return rule_name,rule_value

# APRIORI ALGORITHM FUNCTION

In [12]:
def apriori_algo(data,sup):
    dat=0

    #K==1
    dat=calculate_frequent_items(data,sup)
    print("FREQUENT ITEMS FOR K == 1")
    print(dat)

    #K==2
    return Generalization(dat,sup)

# MULTIPLE MINIMUM SUPPORT FUNCTION WITH CONFIDENCE OF 80%

In [13]:
def MIS(trans_df,sup):
    freqItems=apriori_algo(trans_df,sup)
    k=len(freqItems[0].split("/"))
    print("RULES FOR K == ",k)
    name,val=confidence(freqItems,80,k)
    for j in range(0,len(name)):
        print(name[j]," : ",val[j])

In [14]:
start_time=time.time()
MIS(trans_df,minimum_item_supports)
end_time=time.time()

FREQUENT ITEMS FOR K == 1
['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'HAND WARMER UNION JACK', 'ASSORTED COLOUR BIRD ORNAMENT', "POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN", 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'HOME BUILDING BLOCK WORD', 'LOVE BUILDING BLOCK WORD', 'DOORMAT NEW ENGLAND', 'JAM MAKING SET WITH JARS', 'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE GREEN', 'VINTAGE HEADS AND TAILS CARD GAME ', 'SET/2 RED RETROSPOT TEA TOWELS ', 'ROUND SNACK BOXES SET OF4 WOODLAND ', 'SPACEBOY LUNCH BOX ', 'LUNCH BOX I LOVE LONDON', 'RED TOADSTOOL LED NIGHT LIGHT', ' SET 2 TEA TOWELS I LOVE LONDON ', 'POSTAGE', "PAPER CHAIN KIT 50'S CHRISTMAS ", 'RETRO COFFEE MUGS ASSORTED', 'SAVE THE PLANET MUG', 'WOOD 2 DRAWER CABINET WHITE FINISH', 'WOOD S/3 CABINET ANT WHITE FINISH', 'WOODEN PICTURE FR

In [15]:
end_time-start_time

153.06080222129822